In [227]:
import numpy as np
import pandas as pd

In [228]:
letters = pd.read_csv("letters2.csv")
letters.set_index('letter',inplace = True)
letters["bias"] = 1
letters.head(3)

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,bias
letter,,,,,,,,,,,,,,,,,,,,,
A,0,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
B,1,1,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
C,0,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [229]:
letters.shape

(26, 65)

In [230]:
def in_target(letter:str):
    if ord('A') <= ord(letter) <= ord('I'):
        return 1
    elif ord('J') <= ord(letter) <= ord('R'): 
        return 0
    elif ord('S') <= ord(letter) <= ord('Z'):
        return -1

In [231]:
#אימון הרשת, אין_טארגט נותן את הטווח הרצוי של האות, קרי המטרה
def train_Hebbian(train,alpha =1):
    W = np.zeros(train.shape[1])
    for letter in train.index:
        W = W + np.dot(train.loc[letter],in_target(letter))
    return W

In [232]:
W = train_Hebbian(letters)
W_8x8 = W[:-1].reshape(8, 8)
print("the final Weights vector as 8x8 is:\n\n",W_8x8)
print("the Bias is :",W[-1])

the final Weights vector as 8x8 is:

 [[-2.  3.  4.  5.  5.  3.  1. -4.]
 [ 2. -1.  1.  0.  0.  1.  0. -2.]
 [ 4.  0. -2.  0. -1. -2.  2. -1.]
 [ 6.  5.  3.  0.  2.  3.  4.  0.]
 [ 6.  0. -4. -1.  0. -3.  3. -1.]
 [ 5. -3.  0. -2. -1.  1. -1.  1.]
 [ 3.  3.  3.  0.  3.  2.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
the Bias is : 1.0


### notice the bias is 1!!!!

In [233]:
def test_Hebbian(test,W):
    success = 0
    for letter in test.index:
        fire = np.dot(test.loc[letter],W)
        target = in_target(letter)

        if target == fire == 0: #both are zero - J-R class
            success += 1 
        elif 0 < target <= fire: #both are positive - A-I class
            success += 1
        elif fire <= target < 0: #both are negative - S-Z class
            success += 1
            
    return success*100/26

In [234]:
result = test_Hebbian(letters,W)
print("the success rate of the network is : {:.2f}%".format(result))

the success rate of the network is : 46.15%


In [235]:
mod_letters = letters.copy().replace(0, -1)
mod_letters.head(3)

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,bias
letter,,,,,,,,,,,,,,,,,,,,,
A,-1,-1,-1,1,1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
B,1,1,1,1,1,1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
C,-1,-1,1,1,1,1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [236]:
W_mod = train_Hebbian(mod_letters)
result_mod = test_Hebbian(mod_letters, W_mod)
print("the success rate of the modded network is : {:.2f}%".format(result_mod))

the success rate of the modded network is : 57.69%


# Anomalies!!!!!

In [237]:
def add_noise(data, noise_percent):
    """
    Add random noise to an entire dataset of binary data
    
    Parameters:
    data: pandas DataFrame of 1s and -1s
    noise_percent: float between 0 and 100, percent of bits to flip per row
    
    Returns:
    noisy_data: pandas DataFrame with noise added to each row
    """
    noisy_data = data.copy()
    num_pixels = len(data.columns)
    num_to_flip = int(num_pixels * noise_percent / 100)
    
    # For each row in the dataset
    for idx in data.index:
        # Randomly select indices to flip
        flip_indices = np.random.choice(num_pixels, num_to_flip, replace=False)
        
        # Flip the selected bits (-1->1, 1->-1)
        for flip_idx in flip_indices:
            noisy_data.loc[idx, data.columns[flip_idx]] = -noisy_data.loc[idx, data.columns[flip_idx]]
            
    return noisy_data

# Example usage:
# noisy_A = add_noise(letters.loc['A'], 10)  # Add 10% noise to letter A


In [240]:
W_mod = train_Hebbian(mod_letters)
print("results of noisy data with dataset of 1's and -1's:\n")
for i in [5,10,20,30]:
    per_sum =0
    for j in range(100):
        noisy = add_noise(mod_letters,i)
        #letters.set_index
        ret = test_Hebbian(noisy,W_mod)
        per_sum+=ret/100
    print(f"for noise{i} the result is:  ",per_sum)

results of noisy data with dataset of 1's and -1's:

for noise5 the result is:   55.749999999999986
for noise10 the result is:   55.865384615384635
for noise20 the result is:   54.26923076923079
for noise30 the result is:   51.96153846153848


In [239]:
W = train_Hebbian(letters)
print("results of noisy data with dataset of 1's and 0's:\n")
for i in [5,10,20,30]:
    per_sum =0
    for j in range(200):
        noisy = add_noise(letters,i)
        #letters.set_index
        ret = test_Hebbian(noisy,W)
        per_sum+=ret/200
    print(f"for noise{i} the result is:  ",per_sum)

results of noisy data with dataset of 1's and 0's:

for noise5 the result is:   92.88461538461546
for noise10 the result is:   93.00000000000006
for noise20 the result is:   91.42307692307683
for noise30 the result is:   91.23076923076918


# Making the 64*26 NetWork  


In [16]:
letters.shape

(26, 65)

In [17]:
def to_num(letter):
    # Convert letter to corresponding number (A=1, B=2, ..., Z=26)
    target = ord(letter) - ord('A') + 1
    return target


In [18]:
def train_26_Hebb(train,alpha =26):
    W = np.zeros((train.shape[0],train.shape[1]))
    for weight in range(26):
        for letter in train.index:
            if weight+1 == to_num(letter):
                W[weight] = W[weight] + np.dot(train.loc[letter],alpha)
            else: W[weight] = W[weight] - train.loc[letter]
    return W

In [19]:
def test_26_Hebb(test,W):
    results = [[] for _ in range(26)]  # Initialize list of 26 empty lists
    for weight in range(26):
        for letter in test.index:
            ans = np.dot(W[weight],test.loc[letter])
            if ans > 1:
                results[weight].append(1)
            else:
                results[weight].append(0)
    return results

In [117]:
def test_accuracy(test_results):
    # Create the target array where each letter should only activate its corresponding neuron
    target = np.zeros((26, 26))
    for i in range(26):
        target[i][i] = 1

    # Convert test_results to numpy array for comparison
    results = np.array(test_results)

    # Calculate accuracy by comparing with target array
    correct = 0
    total = 26 * 26  # Total number of predictions

    for i in range(26):
        for j in range(26):
            if results[i][j] == target[i][j]:
                correct += 1

    accuracy = (correct / total) * 100
    print(f"Accuracy: {accuracy:.2f}%")

    # Print where the network made mistakes
    for i in range(26):
        letter = chr(65 + i)  # Convert index to ASCII letter
        mistakes = []
        for j in range(26):
            if results[i][j] != target[i][j]:
                mistakes.append(chr(65 + j))
        if mistakes:
            print(f"Neuron {letter} made mistakes on letters: {', '.join(mistakes)}")
    
    return accuracy

In [21]:
def test_accuracy_simple(test_results):
    target = np.zeros((26, 26))
    for i in range(26):
        target[i][i] = 1

    results = np.array(test_results)
    correct = 0
    total = 26 * 26  # Total number of predictions becuase for each letter success or fail on each letter

    for i in range(26):
        for j in range(26):
            if results[i][j] == target[i][j]:
                correct += 1

    accuracy = (correct / total) * 100
    return accuracy


In [116]:
for i in range(11,16):
    W26 = train_26_Hebb(letters,alpha = i)
    TW26 = test_26_Hebb(letters,W26)
    print("i is:    ",i)
    print(test_accuracy_simple(TW26))

i is:     11
97.92899408284023
i is:     12
98.37278106508876
i is:     13
98.66863905325444
i is:     14
95.11834319526628
i is:     15
92.89940828402366


In [130]:
W26_13 = train_26_Hebb(letters,alpha=13)
TW26_13 = test_26_Hebb(letters,W26_13)
print("dataset of 1's and 0's with alpha = 13\n")
test_accuracy(TW26_13)

dataset of 1's and 0's with alpha = 13

Accuracy: 98.67%
Neuron C made mistakes on letters: C
Neuron F made mistakes on letters: F
Neuron G made mistakes on letters: J
Neuron I made mistakes on letters: T
Neuron L made mistakes on letters: L
Neuron M made mistakes on letters: N
Neuron N made mistakes on letters: M
Neuron P made mistakes on letters: P
Neuron Q made mistakes on letters: O


98.66863905325444

In [125]:
for i in range(9,13):
    W26 = train_26_Hebb(mod_letters,alpha = i)
    TW26 = test_26_Hebb(mod_letters,W26)
    print("i is:    ",i)
    print(test_accuracy_simple(TW26))

i is:     9
97.92899408284023
i is:     10
98.22485207100591
i is:     11
98.66863905325444
i is:     12
96.89349112426035


In [131]:
W26_11 = train_26_Hebb(mod_letters,alpha=11)
TW26_11 = test_26_Hebb(mod_letters,W26_11)
print("dataset of 1's and -1's with alpha = 11\n")
test_accuracy(TW26_11)

dataset of 1's and -1's with alpha = 11

Accuracy: 98.67%
Neuron C made mistakes on letters: C
Neuron F made mistakes on letters: F
Neuron I made mistakes on letters: T
Neuron L made mistakes on letters: L
Neuron M made mistakes on letters: N
Neuron N made mistakes on letters: M
Neuron O made mistakes on letters: Q
Neuron P made mistakes on letters: P
Neuron T made mistakes on letters: I


98.66863905325444

### Noise? what? I cant Hear you?

In [257]:
print("results of noisy data with dataset of 1's and 0's;  alpha = 13:\n")
for i in [5,10,20,30,50]:
    per_sum =0
    for j in range(100):
        noisy = add_noise(letters,i)
        #letters.set_index
        ret = test_26_Hebb(noisy,W26_13)
        per_sum+= test_accuracy_simple(ret)/100
    print(f"for noise{i} the result is:  ",per_sum)


results of noisy data with dataset of 1's and 0's;  alpha = 13:

for noise5 the result is:   97.96301775147928
for noise10 the result is:   97.33136094674559
for noise20 the result is:   94.0991124260355
for noise30 the result is:   88.353550295858
for noise50 the result is:   52.575443786982234


In [258]:
print("results of noisy data with dataset of 1's and -1's;  alpha = 11:\n")
for i in [5,10,20,30,50]:
    per_sum =0
    for j in range(100):
        noisy = add_noise(mod_letters,i)
        #letters.set_index
        ret = test_26_Hebb(noisy,W26_11)
        per_sum+= test_accuracy_simple(ret)/100
    print(f"for noise{i} the result is:  ",per_sum)


results of noisy data with dataset of 1's and -1's;  alpha = 11:

for noise5 the result is:   98.0118343195267
for noise10 the result is:   97.53402366863914
for noise20 the result is:   95.87130177514794
for noise30 the result is:   92.0695266272189
for noise50 the result is:   52.62869822485207


# Making the 64*3 Network

In [138]:
#letters = mod_letters
print(letters.shape)
letters.head(1)


(26, 65)


,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,bias
letter,,,,,,,,,,,,,,,,,,,,,
A,-1,-1,-1,1,1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [139]:
def in_target(letter:str):
    if ord('A') <= ord(letter) <= ord('I'):
        return 1
    elif ord('J') <= ord(letter) <= ord('R'): 
        return 0
    elif ord('S') <= ord(letter) <= ord('Z'):
        return -1

In [140]:
def train_3_Hebb(train,alpha=2):
    W = np.zeros((3,train.shape[1]))
    for weight in range(3):
        for letter in train.index:
            if weight-1 == in_target(letter):
                W[weight] = W[weight] + alpha * train.loc[letter] # Fixed dot product to scalar multiplication
            else:
                W[weight] = W[weight] - train.loc[letter]
    return W[::-1]
#W: [[S-Z],[J-R],[A-I]] ____  W[::-1] = [[A-I],[J-R],[S-Z]]

In [141]:
def test_3_Hebb(test,W):
    results = [[] for _ in range(3)]  # Initialize list of 3 empty lists
    for weight in range(3):
        for letter in test.index:
            ans = np.dot(W[weight],test.loc[letter])
            if ans > 1:
                results[weight].append(1)
            else:
                results[weight].append(0)
    return results
#results[[A,B,C,,,Z]--[A-I], [A,B,C,,,Z]--[J-R], [A,B,C,,,Z]--[S-Z]]

In [142]:
# Find optimal alpha by testing different values
best_alpha = 1
best_accuracy = 0

for alpha in range(1, 101):  # Test alpha values from 1 to 100
    W3 = train_3_Hebb(letters, alpha)
    results = test_3_Hebb(letters, W3)
    
    # Calculate accuracy for this alpha
    correct = 0
    total = len(letters)
    
    for i, letter in enumerate(letters.index):
        target = in_target(letter)
        predicted = None
        
        if results[0][i] == 1 and results[1][i] == 0 and results[2][i] == 0:
            predicted = 1
        elif results[0][i] == 0 and results[1][i] == 1 and results[2][i] == 0:
            predicted = 0
        elif results[0][i] == 0 and results[1][i] == 0 and results[2][i] == 1:
            predicted = -1
            
        if predicted == target:
            correct += 1
    
    accuracy = (correct / total) * 100
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_alpha = alpha

print(f"Best alpha: {best_alpha}")
print(f"Best accuracy: {best_accuracy}%")


Best alpha: 2
Best accuracy: 38.46153846153847%


In [144]:
print("the weights Matrix of the network (65 is 64 pixels + bias)\n")
# Train the network
W3 = train_3_Hebb(letters)
print(W3.shape)
# Test the network
results = test_3_Hebb(letters, W3)

for i in W3:
    print(i)


the weights Matrix of the network (65 is 64 pixels + bias)

(3, 65)
[ -7.  11.  13.  17.  17.   9.   3. -11.   1.  -7.   3.   1.  -1.   3.
  -7.  -7.   5.   1.  -9.   1.  -5.  -7.   3.  -9.  11.  15.  11.   1.
   9.  11.  13.  -5.  11.  -1.  -9.  -1.  -9.  -7.   9.  -7.   9.  -9.
  -1.  -3.  -3.  -5.  -3.   1.   5.  11.   9.   3.   9.   5.  -5.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   1.]
[-1. -7. -5. -7. -7. -3. -3. -5.  7.  5. -3. -5. -1. -3. 11. -1. 11. -5.
  3. -5.  1. -1.  3.  9. 11. -3. -7. -5. -9. -7. -5.  7. 11. -1. -9. -7.
 15. -7. -3.  5.  9. -3. -1. -9. -3. 13. -3.  1.  5. -7. -3. -9. -3. -1.
 13. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.]
[  8.  -4.  -8. -10. -10.  -6.   0.  16.  -8.   2.   0.   4.   2.   0.
  -4.   8. -16.   4.   6.   4.   4.   8.  -6.   0. -22. -12.  -4.   4.
   0.  -4.  -8.  -2. -22.   2.  18.   8.  -6.  14.  -6.   2. -18.  12.
   2.  12.   6.  -8.   6.  -2. -10.  -4.  -6.   6.  -6.  -4.  -8.   2.
   2.   2.   2.   2.   2.   2.   2.   2.  -2.]


In [148]:
def calculate_accuracy(results, letters):
    correct = 0
    total = len(letters)

    for i, letter in enumerate(letters.index):
        target = in_target(letter)
        predicted = None

        # Check which output neuron fired (1) and others didn't (0)
        if results[0][i] == 1 and results[1][i] == 0 and results[2][i] == 0:
            predicted = 1  # First group (A-I)
        elif results[0][i] == 0 and results[1][i] == 1 and results[2][i] == 0:
            predicted = 0  # Second group (J-R) 
        elif results[0][i] == 0 and results[1][i] == 0 and results[2][i] == 1:
            predicted = -1  # Third group (S-Z)

        if predicted == target:
            correct += 1

    accuracy = (correct / total) * 100
    return accuracy



In [149]:
W3 = train_3_Hebb(letters)
results = test_3_Hebb(letters, W3)


W3_mod = train_3_Hebb(mod_letters)
results_mod = test_3_Hebb(mod_letters, W3_mod)

accuracy = calculate_accuracy(results, letters)
accuracy_mod = calculate_accuracy(results_mod, mod_letters)

print(f"\nAccuracy of 1's and 0's: {accuracy:.2f}%")
print(f"\nAccuracy of 1's and -1's: {accuracy_mod:.2f}%")


Accuracy of 1's and 0's: 38.46%

Accuracy of 1's and -1's: 38.46%


In [245]:
print("Base Categories")
print("results of noisy data with dataset of 1's and 0's:\n")
for i in [5,10,20,30]:
    per_sum =0
    for j in range(100):
        noisy = add_noise(letters,i)
        #letters.set_index
        ret = test_3_Hebb(noisy,W3)
        per_sum+= calculate_accuracy(ret,letters)/100
    print(f"for noise{i} the result is:  ",per_sum)

Base Categories
results of noisy data with dataset of 1's and 0's:

for noise5 the result is:   41.96153846153843
for noise10 the result is:   39.3076923076923
for noise20 the result is:   35.57692307692306
for noise30 the result is:   32.46153846153847


In [246]:
print("Base Categories")
print("results of noisy data with dataset of 1's and -1's:\n")
for i in [5,10,20,30]:
    per_sum =0
    for j in range(100):
        noisy = add_noise(mod_letters,i)
        #letters.set_index
        ret = test_3_Hebb(noisy,W3_mod)
        per_sum+= calculate_accuracy(ret,mod_letters)/100
    print(f"for noise{i} the result is:  ",per_sum)

Base Categories
results of noisy data with dataset of 1's and -1's:

for noise5 the result is:   34.76923076923077
for noise10 the result is:   34.615384615384606
for noise20 the result is:   36.038461538461526
for noise30 the result is:   34.53846153846155


# New World Order

In [195]:
def in_target2(letter:str):
    first_group = ['I', 'L', 'T', 'F', 'E', 'H', 'N', 'M']
    second_group = ['O', 'Q', 'C', 'G', 'D', 'B', 'P', 'R', 'J']
    third_group = ['A', 'V', 'W', 'X', 'Y', 'Z', 'K', 'S', 'U']
    
    if letter in first_group:
        return 1
    elif letter in second_group:
        return 0
    elif letter in third_group:
        return -1

In [196]:
def train_3_Hebb2(train,alpha=2):#test is the same function
    W = np.zeros((3,train.shape[1]))
    for weight in range(3):
        for letter in train.index:
            if weight-1 == in_target2(letter):
                W[weight] = W[weight] + alpha * train.loc[letter] # Fixed dot product to scalar multiplication
            else:
                W[weight] = W[weight] - train.loc[letter]
    return W[::-1]

In [197]:
def calculate_accuracy2(results, letters):
    correct = 0
    total = len(letters)

    for i, letter in enumerate(letters.index):
        target = in_target2(letter)
        predicted = None

        # Check which output neuron fired (1) and others didn't (0)
        if results[0][i] == 1 and results[1][i] == 0 and results[2][i] == 0:
            predicted = 1  # First group (A-I)
        elif results[0][i] == 0 and results[1][i] == 1 and results[2][i] == 0:
            predicted = 0  # Second group (J-R) 
        elif results[0][i] == 0 and results[1][i] == 0 and results[2][i] == 1:
            predicted = -1  # Third group (S-Z)

        if predicted == target:
            correct += 1

    accuracy = (correct / total) * 100
    return accuracy

In [198]:
W3_2 = train_3_Hebb2(letters)
results_2 = test_3_Hebb(letters, W3_2)

W3_mod_2 = train_3_Hebb2(mod_letters)
results_mod_2 = test_3_Hebb(mod_letters, W3_mod_2)


accuracy_2 = calculate_accuracy2(results_2, letters)
accuracy_mod_2 = calculate_accuracy2(results_mod_2, mod_letters)

print(f"\nAccuracy of 1's and 0's: {accuracy_2:.2f}%")
print(f"\nAccuracy of 1's and -1's: {accuracy_mod_2:.2f}%")


Accuracy of 1's and 0's: 53.85%

Accuracy of 1's and -1's: 50.00%


In [247]:
print("New Categories")
print("results of noisy data with dataset of 1's and 0's:\n")
for i in [5,10,20,30]:
    per_sum =0
    for j in range(100):
        noisy = add_noise(letters,i)
        #letters.set_index
        ret = test_3_Hebb(noisy,W3_2)
        per_sum+= calculate_accuracy2(ret,letters)/100
    print(f"for noise{i} the result is:  ",per_sum)

New Categories
results of noisy data with dataset of 1's and 0's:

for noise5 the result is:   53.34615384615389
for noise10 the result is:   51.961538461538495
for noise20 the result is:   46.769230769230745
for noise30 the result is:   41.38461538461537


In [248]:
print("New Categories")
print("results of noisy data with dataset of 1's and -1's:\n")
for i in [5,10,20,30]:
    per_sum =0
    for j in range(100):
        noisy = add_noise(mod_letters,i)
        #letters.set_index
        ret = test_3_Hebb(noisy,W3_mod_2)
        per_sum+= calculate_accuracy2(ret,mod_letters)/100
    print(f"for noise{i} the result is:  ",per_sum)

New Categories
results of noisy data with dataset of 1's and -1's:

for noise5 the result is:   44.499999999999936
for noise10 the result is:   46.38461538461535
for noise20 the result is:   46.038461538461526
for noise30 the result is:   41.15384615384613
